# Trabalho Prático 1 


## Grupo 04 - Renato Garcia (A101987) & Bernardo Moniz (A102497)

## Problema 1
### Enunciado

1. Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

   a. Cada reunião ocupa uma sala (enumeradas $1...S\,$) durante um “slot” $(\text{hora},\text{dia})$.  Assume-se os dias enumerados $1..D$ e, em cada dia, os tempos enumerados $1..T$.

   b. Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.

   c. Cada projeto tem associado um conjunto de colaboradores, dos quais um é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.

   d. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de $50\%$ do total de colaboradores do projeto. A disponibilidade de cada participante, incluindo o líder, é um conjunto de “slots” (“inputs” do problema).


### Resolução

Antes de começar explicitamente o código, temos de importar as bibliotecas necessárias para resolver o exercício abaixo:
- o pywraplp é um solver usado para resolver exercícios de programação linear;
- a função tabulate é utilizada para criar tabelas a partir de dados em Python;
- o módulo random é usado para fornecer funções que geram números aleatórios.

In [1]:
from ortools.linear_solver import pywraplp
from tabulate import tabulate
import random

A função create_slots cria as slots para a alocação de reuniões e recebe como parâmetros os seguintes elementos:
- D: representa os dias;
- duracao_reuniao: representa a duração em minutos de cada reunião;
- hora_inicio: representa a hora de início de um dia;
- hora_fim: representa a hora em que termina o dia;

In [2]:
def create_slots(D, duracao_reuniao, hora_inicio, hora_fim):
    inicio_min = int(hora_inicio.split(':')[0]) * 60 + int(hora_inicio.split(':')[1])
    fim_min = int(hora_fim.split(':')[0]) * 60 + int(hora_fim.split(':')[1])
    almoco_inicio = 12 * 60 + 30  
    almoco_fim = 14 * 60          

    slots = []
    for d in range(1, D + 1):
        hora_atual = inicio_min
        while hora_atual + duracao_reuniao <= fim_min:
            if (hora_atual + duracao_reuniao <= almoco_inicio) or (hora_atual >= almoco_fim):
                hora_inicio = f"{hora_atual // 60:02d}:{hora_atual % 60:02d}"
                hora_fim = f"{(hora_atual + duracao_reuniao) // 60:02d}:{(hora_atual + duracao_reuniao) % 60:02d}"
                slots.append((d, hora_inicio, hora_fim, duracao_reuniao))
            hora_atual = almoco_fim if almoco_inicio <= hora_atual < almoco_fim else hora_atual + duracao_reuniao

    return slots

A função schedule_meetings recebe os seguintes argumentos:

- S: representa o número de salas disponíveis para reuniões;
- P: representa o número de projetos que existem;
- C: representa o número de colaboradores existentes;
- D: estabelece o número de dias;
- assoc_projeto_colaboradores: representa um dicionário que tem como chave o projeto a realizar e como valor os colaboradores alocados a um projeto p;
- lideres_projetos é um dicionário que associa a cada projeto o seu respetivo líder;
- reunioes_semanais: representa o número de reuniões a realizar para cada projeto;
- duracao_reuniao: representa a duração em minutos de cada reunião;
- hora_inicio: representa a hora de início de um dia;
- hora_fim: representa a hora em que termina o dia;
- slots: representa as slots disponíveis ao longo de cada dia;
- disp: é uma lista de listas que retorna a disponibilidade de cada colaborador em todas as slots, gerada de forma aleatória.
  
As matrizes Q e R representam matrizes de alocação (reuniões e alocação de colaboradores a reuniões, respetivamente).

A partir daí, temos as restrições;

- o colaborador c só pode estar presente se o projeto p estiver a ocorrer naquela sala s, dia d e slot t. Para além disso, isso só acontecerá se o colaborador estiver associado ao projeto p;
- um colaborador c só pode participar numa reunião se ele estiver disponível naquele horário;
- para cada colaborador c, temos de garantir que a soma de todas as participações em todos as salas e projetos é igual 1, ou seja, o colaborador c só pode estar no máximo numa reunião naquele slot t;
- nesta restrição garantimos que só existe uma reunião, no máximo, por sala;
- nesta restrição garantimos que pelo menos 50% dos colaboradores alocados a um projeto têm de estar presentes para se realizar uma reunião. Esta só se realizará se estiver garantido que o líder do projeto correspondente a essa reunião estiver presente na mesma;
- esta restrição garante que o colaborador c está presente na reunião do projeto p na qual é líder;
- a última restrição garante que cada projeto tenha pelo menos o número mínimo de reuniões semanais exigido.

Para além das restrições, usamos ainda um "solver.Maximize()" para maximizar o número de reuniões por projetos.

Por fim, utilizamos a função tabulate para printar o horário final.


In [3]:
def schedule_meetings(S, P, C, D, assoc_projeto_colaboradores, lideres_projetos, reunioes_semanais, duracao_reuniao, hora_inicio, hora_fim, slots,disp): 
    solver = pywraplp.Solver.CreateSolver('SCIP')

    Q = {}
    R = {}
    for p in range(1, P + 1):
        Q[p] = {}
        for s in range(1, S + 1):
            Q[p][s] = {}
            for d in range(1, D + 1):
                Q[p][s][d] = {}
                for t in range(len(slots)):
                    if slots[t][0] == d:
                        Q[p][s][d][t] = solver.BoolVar(f'Q[{p}],[{s}],[{d}],[{t}]')

    for c in range(1, C + 1):
        R[c] = {}
        for p in range(1, P + 1):
            R[c][p] = {}
            for s in range(1, S + 1):
                R[c][p][s] = {}
                for d in range(1, D + 1):
                    R[c][p][s][d] = {}
                    for t in range(len(slots)):
                        if slots[t][0] == d:
                            R[c][p][s][d][t] = solver.BoolVar(f'R[{c}],[{p}],[{s}],[{d}],[{t}]')

    for d in range(1, D + 1):
        for t in range(len(slots)):
            if slots[t][0] == d:
                for s in range(1, S + 1):
                    for p in range(1, P + 1):
                        for c in range(1, C + 1):
                            solver.Add(R[c][p][s][d][t] <= Q[p][s][d][t] * (1 if c in assoc_projeto_colaboradores[p] else 0))
    
    for c in range(1, C + 1):
        for p in range(1, P + 1):
            for s in range(1, S + 1):
                for d in range(1, D + 1):
                    for t in range(len(slots)):
                        if slots[t][0] == d: 
                            solver.Add(R[c][p][s][d][t] <= disp[c - 1][t]) 

    for c in range(1, C + 1):
        for d in range(1, D + 1):
            for t in range(len(slots)):
                if slots[t][0] == d:
                    solver.Add(sum(R[c][p][s][d][t] for s in range(1, S + 1) for p in range(1, P + 1)) <= 1)

    for s in range(1, S + 1):
        for d in range(1, D + 1):
            for t in range(len(slots)):
                if slots[t][0] == d:
                    solver.Add(sum(Q[p][s][d][t] for p in range(1, P + 1)) <= 1)

    for p in range(1, P + 1):
        for s in range(1, S + 1):
            for d in range(1, D + 1):
                for t in range(len(slots)):
                    if slots[t][0] == d:
                        solver.Add((sum(R[c][p][s][d][t] for c in range(1, C + 1)) / len(assoc_projeto_colaboradores[p])) >= 0.5 * R[lideres_projetos[p]][p][s][d][t])

    for p in range(1, P + 1):
        for s in range(1, S + 1):
            for d in range(1, D + 1):
                for t in range(len(slots)):
                    if slots[t][0] == d:
                        solver.Add(R[lideres_projetos[p]][p][s][d][t] == Q[p][s][d][t])

    for p in range(1, P + 1):
        solver.Add(sum(Q[p][s][d][t] for s in range(1, S + 1) for d in range(1, D + 1) for t in range(len(slots)) if slots[t][0] == d) >= reunioes_semanais)

    solver.Maximize(sum(Q[p][s][d][t] for p in range(1, P + 1) for s in range(1, S + 1) for d in range(1, D + 1) for t in range(len(slots)) if slots[t][0] == d))

    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        horarios_unicos = sorted(set(slot[1] for slot in slots))
        dias_unicos = sorted(set(slot[0] for slot in slots))
    
        table = [["Horários/Dias"] + [f"Dia {dia}" for dia in dias_unicos]]  
    
        for horario in horarios_unicos:
            row = [horario]
            for dia in dias_unicos:
                cell_content = ""
                for t, slot in enumerate(slots):
                    if slot[0] == dia and slot[1] == horario:
                        for p in range(1, P + 1):
                            for s in range(1, S + 1):
                                if int(Q[p][s][dia][t].solution_value()) == 1:
                                    cell_content += (f"\nProjeto {p}") 
                                    cell_content += f" - Sala {s}"      
                                    cell_content += f"\nLíder: Colaborador {lideres_projetos[p]}"  
                                    cell_content += "\nColaboradores: "
                                    for c in range(1, C + 1):
                                        if int(R[c][p][s][dia][t].solution_value()) == 1:
                                            cell_content += f"{c}  "  
                                    cell_content = cell_content.rstrip()
                        break
                row.append(cell_content)
            table.append(row)
    
        print(tabulate(table, headers="firstrow", tablefmt="grid"))
    else:
        print("No solution found")

#### Exemplo 1

In [4]:
S = 4  
P = 4  
C = 8  
D = 5  
assoc_projeto_colaboradores = {
    1: [1, 2, 3, 4],
    2: [1, 2, 5, 6],
    3: [3, 4, 7, 8],
    4: [5, 6, 7, 8]
}

reunioes_semanais = 4
duracao_reuniao = 70
hora_inicio = "09:00"
hora_fim = "19:00"
slots = create_slots(D, duracao_reuniao, hora_inicio, hora_fim)
disp = [[random.choice([0, 1]) for _ in slots] for _ in range(1, C + 1)]
lideres_projetos = {p: random.choice(colaboradores) for p, colaboradores in assoc_projeto_colaboradores.items()}

schedule_meetings(S, P, C, D, assoc_projeto_colaboradores, lideres_projetos, reunioes_semanais, duracao_reuniao, hora_inicio, hora_fim, slots, disp)

+-----------------+----------------------+----------------------+----------------------+----------------------+----------------------+
| Horários/Dias   | Dia 1                | Dia 2                | Dia 3                | Dia 4                | Dia 5                |
+=================+======================+======================+======================+======================+======================+
| 09:00           | Projeto 2 - Sala 1   | Projeto 1 - Sala 2   | Projeto 1 - Sala 2   | Projeto 3 - Sala 2   | Projeto 1 - Sala 2   |
|                 | Líder: Colaborador 6 | Líder: Colaborador 4 | Líder: Colaborador 4 | Líder: Colaborador 8 | Líder: Colaborador 4 |
|                 | Colaboradores: 2  6  | Colaboradores: 3  4  | Colaboradores: 2  4  | Colaboradores: 4  8  | Colaboradores: 2  4  |
|                 | Projeto 3 - Sala 2   |                      | Projeto 3 - Sala 3   |                      | Projeto 2 - Sala 1   |
|                 | Líder: Colaborador 8 |             

#### Exemplo 2

In [5]:
S = 5  
P = 5  
C = 10  
D = 5  
assoc_projeto_colaboradores = {
    1: [1, 2, 3, 4, 7],
    2: [1, 2, 5, 6, 10],
    3: [3, 4, 8, 9],
    4: [5, 6, 7, 8],
    5: [2, 4, 5, 6]
}

reunioes_semanais = 3
duracao_reuniao = 80 
hora_inicio = "10:00"
hora_fim = "19:00"
slots = create_slots(D, duracao_reuniao, hora_inicio, hora_fim)
disp = [[random.choice([0, 1]) for _ in slots] for _ in range(1, C + 1)]
lideres_projetos = {p: random.choice(colaboradores) for p, colaboradores in assoc_projeto_colaboradores.items()}

schedule_meetings(S, P, C, D, assoc_projeto_colaboradores, lideres_projetos, reunioes_semanais, duracao_reuniao, hora_inicio, hora_fim, slots, disp)

+-----------------+-------------------------+-------------------------+----------------------+-------------------------+------------------------+
| Horários/Dias   | Dia 1                   | Dia 2                   | Dia 3                | Dia 4                   | Dia 5                  |
+=================+=========================+=========================+======================+=========================+========================+
| 10:00           |                         | Projeto 3 - Sala 5      | Projeto 3 - Sala 2   | Projeto 1 - Sala 4      |                        |
|                 |                         | Líder: Colaborador 9    | Líder: Colaborador 9 | Líder: Colaborador 4    |                        |
|                 |                         | Colaboradores: 8  9     | Colaboradores: 3  9  | Colaboradores: 3  4  7  |                        |
|                 |                         |                         | Projeto 4 - Sala 1   | Projeto 2 - Sala 5      |    

#### Exemplo 3 

In [6]:
S = 6  
P = 10  
C = 25  
D = 5  
assoc_projeto_colaboradores = {
    1: [1, 2, 3, 4, 7, 9, 16, 17, 21, 22],
    2: [1, 2, 5, 6, 10, 16, 18, 23, 24],
    3: [3, 4, 8, 9, 11, 12, 18, 17, 25],
    4: [5, 6, 7, 8, 10, 19, 20],
    5: [2, 4, 5, 6, 13, 14, 15],
    6: [2, 3, 9, 13, 14],
    7: [1, 6, 10, 11, 13, 15],
    8: [5, 7, 12, 14, 15, 17, 20],
    9: [4, 8, 9, 16, 18, 21],
    10: [1, 10, 12, 13, 19, 22, 23]
}

reunioes_semanais = 6
duracao_reuniao = 50 
hora_inicio = "08:30"
hora_fim = "18:00"
slots = create_slots(D, duracao_reuniao, hora_inicio, hora_fim)
disp = [[random.choice([0, 1]) for _ in slots] for _ in range(1, C + 1)]
lideres_projetos = {p: random.choice(colaboradores) for p, colaboradores in assoc_projeto_colaboradores.items()}

schedule_meetings(S, P, C, D, assoc_projeto_colaboradores, lideres_projetos, reunioes_semanais, duracao_reuniao, hora_inicio, hora_fim, slots, disp)

+-----------------+---------------------------------+-----------------------------------+----------------------------------+----------------------------------+---------------------------------+
| Horários/Dias   | Dia 1                           | Dia 2                             | Dia 3                            | Dia 4                            | Dia 5                           |
+=================+=================================+===================================+==================================+==================================+=================================+
| 08:30           | Projeto 3 - Sala 5              | Projeto 7 - Sala 1                | Projeto 3 - Sala 2               | Projeto 1 - Sala 2               | Projeto 4 - Sala 1              |
|                 | Líder: Colaborador 17           | Líder: Colaborador 6              | Líder: Colaborador 17            | Líder: Colaborador 2             | Líder: Colaborador 8            |
|                 | Colaborado

#### Exemplo 4

In [7]:
S = 6  
P = 10  
C = 50  
D = 5  
assoc_projeto_colaboradores = {
    1: [1, 2, 3, 4, 7, 9, 16, 17, 21, 22],
    2: [1, 2, 5, 6, 10, 16, 18, 23, 24],
    3: [3, 4, 8, 9, 11, 12, 18, 17, 25],
    4: [5, 6, 7, 8, 10, 19, 20, 26, 27],
    5: [2, 4, 5, 6, 13, 14, 15, 28, 29],
    6: [2, 3, 9, 13, 14, 30, 31],
    7: [1, 6, 10, 11, 13, 15, 32, 33],
    8: [5, 7, 12, 14, 15, 17, 20, 34, 35],
    9: [4, 8, 9, 16, 18, 21, 36, 37],
    10: [1, 10, 12, 13, 19, 22, 23, 38, 39]
}

reunioes_semanais = 8
duracao_reuniao = 45 
hora_inicio = "08:00"
hora_fim = "20:00"
slots = create_slots(D, duracao_reuniao, hora_inicio, hora_fim)
disp = [[random.choice([0, 1]) for _ in slots] for _ in range(1,C+1)]
lideres_projetos = {p: random.choice(colaboradores) for p, colaboradores in assoc_projeto_colaboradores.items()}

schedule_meetings(S, P, C, D, assoc_projeto_colaboradores,lideres_projetos, reunioes_semanais, duracao_reuniao, hora_inicio, hora_fim,slots, disp)

+-----------------+-----------------------------------+-----------------------------------+------------------------------------+-----------------------------------+------------------------------------------+
| Horários/Dias   | Dia 1                             | Dia 2                             | Dia 3                              | Dia 4                             | Dia 5                                    |
+=================+===================================+===================================+====================================+===================================+==========================================+
| 08:00           | Projeto 2 - Sala 5                | Projeto 5 - Sala 3                | Projeto 1 - Sala 2                 | Projeto 2 - Sala 6                | Projeto 9 - Sala 6                       |
|                 | Líder: Colaborador 23             | Líder: Colaborador 13             | Líder: Colaborador 2               | Líder: Colaborador 23             | Líd